Internet Resources:

[Python Programming.net - machine learning episodes 34-38](https://pythonprogramming.net/machine-learning-clustering-introduction-machine-learning-tutorial/)

In [1]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandas as pd

# data from https://pythonprogramming.net/static/downloads/machine-learning-data/titanic.xls
df = pd.read_excel('data/titanic.xls')

'''
Pclass Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
survival Survival (0 = No; 1 = Yes)
name Name
sex Sex
age Age
sibsp Number of Siblings/Spouses Aboard
parch Number of Parents/Children Aboard
ticket Ticket Number
fare Passenger Fare (British pound)
cabin Cabin
embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat Lifeboat
body Body Identification Number
home.dest Home/Destination
'''

df.drop(['body','name'], 1, inplace=True)
df.apply(pd.to_numeric, errors="ignore")
df.fillna(0, inplace=True)
print(df.head())

   pclass  survived     sex      age  sibsp  parch  ticket      fare    cabin  \
0       1         1  female  29.0000      0      0   24160  211.3375       B5   
1       1         1    male   0.9167      1      2  113781  151.5500  C22 C26   
2       1         0  female   2.0000      1      2  113781  151.5500  C22 C26   
3       1         0    male  30.0000      1      2  113781  151.5500  C22 C26   
4       1         0  female  25.0000      1      2  113781  151.5500  C22 C26   

  embarked boat                        home.dest  
0        S    2                     St Louis, MO  
1        S   11  Montreal, PQ / Chesterville, ON  
2        S    0  Montreal, PQ / Chesterville, ON  
3        S    0  Montreal, PQ / Chesterville, ON  
4        S    0  Montreal, PQ / Chesterville, ON  


In [2]:
# handling non linear data:
# every unique feature value gets a different integer number
# e.g. female becomes 0, male becomes 1
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

df = handle_non_numerical_data(df)
print(df.head())

   pclass  survived  sex      age  sibsp  parch  ticket      fare  cabin  \
0       1         1    0  29.0000      0      0     753  211.3375     64   
1       1         1    1   0.9167      1      2     502  151.5500    161   
2       1         0    0   2.0000      1      2     502  151.5500    161   
3       1         0    1  30.0000      1      2     502  151.5500    161   
4       1         0    0  25.0000      1      2     502  151.5500    161   

   embarked  boat  home.dest  
0         0     1         92  
1         0     8        115  
2         0     0        115  
3         0     0        115  
4         0     0        115  


In [3]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X) # https://scikit-learn.org/stable/modules/preprocessing.html
y = np.array(df['survived'])

clf = KMeans(n_clusters=2)
clf.fit(X)

correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me)) # makes double list [x1,x2...] -> [[x1,x2...]]
    prediction = clf.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1
        
# this may not be the actual accuracy because the model knows nothing
# about labels because it just finds clusters in the training data
# so accuracy is either correct/len(X) or 1 - correct/len(X)
print(correct/len(X)) 

0.7119938884644768
